In [ ]:
!wget https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg?raw=true -O yolov3.cfg
!wget https://pjreddie.com/media/files/yolov3.weights
!wget https://github.com/pjreddie/darknet/blob/master/data/coco.names?raw=true -O coco.names

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getUnconnectedOutLayersNames()

In [ ]:
from IPython.display import clear_output, display, Image
import PIL.Image

def showarray(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = PIL.Image.fromarray(a)
    display(Image(data=fmt_formatter(f, fmt)))

def fmt_formatter(data, fmt):
    from IPython.display import display
    import io
    str_buf = io.BytesIO()
    data.save(str_buf, format=fmt)
    return str_buf.getvalue()

def detect_objects(video_path):
    cap = cv2.VideoCapture(video_path)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        height, width, _ = frame.shape
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
       outs = net.forward(layer_names)

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                if confidence > 0.5 and (class_id == 2 or class_id == 0):
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    class_ids.append(class_id)
                    confidences.append(float(confidence))
                    boxes.append([x, y, w, h])

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        for i in range(len(boxes)):
            x, y, w, h = boxes[i]

            if i in indexes:
                if class_ids[i] == 2:  # Car
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                elif class_ids[i] == 0:  # Person (Assuming person as an empty space)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                else:
                    # Assume anything other than car or person as empty space
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

        showarray(frame)

# Replace 'videoplayback.mp4' with your video file path
detect_objects('/content/parkinglot_1_480p.mp4')